# Lateral movement via WMI
https://attack.mitre.org/techniques/T1047/

## Example Data Source
https://securitydatasets.com/notebooks/atomic/windows/lateral_movement/SDWIN-200914080546.html

Let's start by looking for any WMIC activity:

In [7]:
wmic_procs = GET process
             FROM https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_wmic_add_user_backdoor.zip
             WHERE [process:command_line LIKE '%WMIC%']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
wmic_procs,process,26,238,238,346,354,8,4,218,84,18,238,238


We found multiple records of 1 unique process entity.  Let's examine those:

In [8]:
DISP wmic_procs ATTR created, command_line, pid, x_unique_id

created,command_line,pid,x_unique_id
None,None,9428,None
None,"""C:\windows\System32\Wbem\WMIC.exe"" /node:WORKSTATION6 process call create ""net user /add backdoor paw0rd1""",9428,None
2020-09-14T12:06:01.826Z,"""C:\windows\System32\Wbem\WMIC.exe"" /node:WORKSTATION6 process call create ""net user /add backdoor paw0rd1""",9428,{2d351099-5ca9-5f5f-2f04-000000000400}


The command line alone gives this one away: `WMIC.exe` is used to create a new process on `WORKSTATION6`.  Let's try to find those processes by looking for the WMI Provider Host (`WMIPrvSE.exe`):

In [9]:
wmiprvse_procs = GET process
                 WHERE [process:command_line LIKE '%wmiprvse%']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
wmiprvse_procs,process,28,329,364,549,596,0,0,364,98,30,380,380


In [10]:
DISP wmiprvse_procs ATTR created, command_line, pid, x_unique_id

created,command_line,pid,x_unique_id
None,None,3952,None
None,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding,3952,None
2020-09-14T12:06:02.209Z,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding,3952,{83d0c8c3-5caa-5f5f-f002-000000000400}
None,None,7296,None
None,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding,7296,None
2020-09-14T12:06:08.432Z,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding,7296,{2d351099-5cb0-5f5f-3804-000000000400}


So it looks like there's 2 instances, pids 3952 and 7296.  They both occur after the `WMIC.exe` command we saw; the first timestamp is less than a second later.  We can now try to find the processes created by these 2 instances:

In [11]:
spawned = FIND process CREATED BY wmiprvse_procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
spawned,process,6,55,390,576,644,0,0,418,106,30,406,406


In [12]:
DISP spawned ATTR created, command_line, pid, x_unique_id, parent_ref.x_unique_id

created,command_line,pid,x_unique_id,parent_ref.x_unique_id
None,None,7768,None,None
None,net user /add backdoor paw0rd1,7768,None,None
2020-09-14T12:06:02.316Z,net user /add backdoor paw0rd1,7768,{83d0c8c3-5caa-5f5f-f102-000000000400},{83d0c8c3-5caa-5f5f-f002-000000000400}


We only found 1, but it matches the original `WMIC.exe` command line we found initially.  